In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
! pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! kaggle datasets download -d mira318/russian-articles-data

 94% 341M/362M [00:04<00:00, 135MB/s]
100% 362M/362M [00:04<00:00, 89.1MB/s]


In [ ]:
! unzip russian-articles-data.zip

Archive:  russian-articles-data.zip
  inflating: articles_data/articles_part_0.csv  
  inflating: articles_data/articles_part_1.csv  
  inflating: articles_data/articles_part_2.csv  
  inflating: articles_data/articles_part_3.csv  
  inflating: articles_data/articles_part_4.csv  
  inflating: articles_data/articles_part_5.csv  
  inflating: articles_data/articles_part_6.csv  
  inflating: articles_data/articles_part_7.csv  
  inflating: articles_data/articles_part_8.csv  
  inflating: articles_data/query_list.txt  
  inflating: articles_data/visited_set.txt  


In [11]:
! pip install datasets==2.11.0 \
lexrank==0.1.0 \
razdel==0.5.0 \
rouge==1.0.1 \
sentencepiece==0.1.97 \
summa==1.2.0 \
tokenizers==0.13.2 \
transformers==4.27.4 \
fasttext

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from datasets import Dataset, DatasetDict
import razdel
import fasttext

from evaluate import print_metrics, postprocess

## Data preparation

In [2]:
# Loading model for language detection
! wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz
lang_detector = fasttext.load_model('lid.176.ftz')

data_path = '/DATA/ichuviliaeva/project_data'
data = []
for file_name in os.listdir(data_path):
    if 'articles' not in file_name:
        continue
    data_part = pd.read_csv(os.path.join(data_path, file_name))

    # Checking that article and abstract are written in russian
    for article, abstract in tqdm(data_part[['article', 'abstract']].to_numpy()):
        if lang_detector.predict(article.replace('\n', ' '), k=1)[0][0] == '__label__en' or \
            lang_detector.predict(abstract.replace('\n', ' '), k=1)[0][0] == '__label__en':
            continue
        data.append([article, abstract])

data = pd.DataFrame(data, columns=['article', 'abstract'])

data.head()

--2023-04-10 16:03:47--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.173.233.83, 18.173.233.68, 18.173.233.125, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.173.233.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 938013 (916K) [binary/octet-stream]
Saving to: ‘lid.176.ftz.2’

lid.176.ftz.2       100%[===================>] 916.03K  2.34MB/s    in 0.4s    

2023-04-10 16:03:48 (2.34 MB/s) - ‘lid.176.ftz.2’ saved [938013/938013]



  0%|          | 0/800 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

...

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

,article,abstract
0,Состав и калорийность\nПлоды айвы достаточно н...,"Айву, которую часто называют ложным яблоком вс..."
1,В последние годы при госпитализации больных с ...,Исследовалась частота ассоциации заболеваний щ...
2,"Введение. Одной из серьезных проблем, стоящих ...",Цель исследования - провести анализ динамики с...
3,Актуальность проблемы. На современном этапе пр...,В статье проанализированы данные о впервые выя...
4,Сохранение здоровья детей в Российской Федерац...,В статье представлены краткие данные о распрос...


In [3]:
print(len(data) - len(data.drop_duplicates()))

579


In [4]:
data.drop_duplicates(inplace=True)

In [5]:
dataset = Dataset.from_pandas(data)
dataset = dataset.train_test_split(test_size=0.2, seed=42)
test_val_dataset = dataset['test'].train_test_split(test_size=0.5, seed=42)

datasets = DatasetDict({
    'train': dataset['train'],
    'valid': test_val_dataset['train'],
    'test': test_val_dataset['test']
})
datasets

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract', '__index_level_0__'],
        num_rows: 30476
    })
    valid: Dataset({
        features: ['article', 'abstract', '__index_level_0__'],
        num_rows: 3810
    })
    test: Dataset({
        features: ['article', 'abstract', '__index_level_0__'],
        num_rows: 3810
    })
})

In [6]:
datasets['train'].to_json('/DATA/ichuviliaeva/project_data/train_data.json')
datasets['valid'].to_json('/DATA/ichuviliaeva/project_data/val_data.json')
datasets['test'].to_json('/DATA/ichuviliaeva/project_data/test_data.json')

Creating json from Arrow format:   0%|          | 0/31 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

357002064

In [ ]:
data['abstract'].apply(lambda x: len(x)).describe()

count    38096.000000
mean       780.029006
std        624.751188
min         28.000000
25%        343.000000
50%        558.000000
75%       1031.250000
max       9454.000000
Name: abstract, dtype: float64

## Extractive methods

In [ ]:
def calc_method_score(records, predict_func):
    refs = []
    preds = []
    for i, record in enumerate(records):
        refs.append(record['abstract'])
        preds.append(predict_func(record['article'], record['abstract']))

    for i, (ref, pred) in enumerate(zip(refs, preds)):
        refs[i], preds[i] = postprocess(ref, pred, tokenize_after=True, lower=True)

    print_metrics(refs, preds)


### LexRank

In [ ]:
import lexrank
from lexrank import LexRank
from lexrank.mappings.stopwords import STOPWORDS
import sys


def predict_lex_rank(text, summary, lxr, summary_size=3, threshold=None):
    sentences = [s.text for s in razdel.sentenize(text)]
    prediction = lxr.get_summary(sentences, summary_size=summary_size, threshold=threshold)
    prediction = " ".join(prediction)

    recursion_depth = len(prediction) * len(summary) + 10
    if sys.getrecursionlimit() < recursion_depth:
        sys.setrecursionlimit(recursion_depth)
    return prediction

In [ ]:
sentences = [[s.text for s in razdel.sentenize(article)] for article in datasets['test']['article']]
lxr = LexRank(sentences, stopwords=STOPWORDS['ru'])
calc_method_score(datasets['test'], lambda x, y: predict_lex_rank(x, y, lxr))

-------------METRICS-------------
Count:	 3868
Ref:	 исследованы размерно-весовой и возрастной состав , а также продукционные характеристики 40 наиболее массовых и обычных видов рыб эстуариев зал . петра великого . установлено , что в эстуариях преобладают короткоцикловые высокопродуктивные виды . максимальный возраст рыб в выборках изменялся от 1 года ( лапша-рыба salangichthys microdon ) до 9 лет ( полосатая камбала liopsetta pinnifasciata ) . удельная годовая продукция рыб варьировала от 0,76 до 10,88 год-1 ( в среднем 1,57 ± 0,25 год-1 ) , для большинства видов значение удельной продукции укладывалось в интервал 0,8-1,2 год-1 . доля продукции рыб в возрасте до одного года в общей продукции составляла 15-100 % ( в среднем 44,0 ± 3,2 % ) и закономерно снижалась по мере увеличения продолжительности жизни . показана взаимозависимость таких параметров популяций , как удельная продукция , естественная смертность , продолжительность жизни и средняя масса особей . трофическая структура соо

### TextRank

In [ ]:
from summa.summarizer import summarize


def predict_text_rank(text, summary, summary_part=0.1):
    prediction = summarize(text, ratio=summary_part, language='russian').replace("\n", " ")

    recursion_depth = len(prediction) * len(summary) + 10
    if sys.getrecursionlimit() < recursion_depth:
        sys.setrecursionlimit(recursion_depth)
    return prediction

In [ ]:
# Tokenizing texts for correct work of TextRank algorithm
abstracts = datasets['test']['abstract']
articles = datasets['test']['article']
for i in range(len(abstracts)):
   abstracts[i] = ' '.join([x.text for x in razdel.tokenize(abstracts[i])])
   articles[i] = ' '.join([x.text for x in razdel.tokenize(articles[i])])

In [ ]:
calc_method_score(
    Dataset.from_dict({'article': articles, 'abstract': abstracts}), 
    lambda x, y: predict_text_rank(x, y, summary_part=0.01)
)

-------------METRICS-------------
Count:	 3868
Ref:	 исследованы размерно-весовой и возрастной состав , а также продукционные характеристики 40 наиболее массовых и обычных видов рыб эстуариев зал . петра великого . установлено , что в эстуариях преобладают короткоцикловые высокопродуктивные виды . максимальный возраст рыб в выборках изменялся от 1 года ( лапша-рыба salangichthys microdon ) до 9 лет ( полосатая камбала liopsetta pinnifasciata ) . удельная годовая продукция рыб варьировала от 0,76 до 10,88 год-1 ( в среднем 1,57 ± 0,25 год-1 ) , для большинства видов значение удельной продукции укладывалось в интервал 0,8-1,2 год-1 . доля продукции рыб в возрасте до одного года в общей продукции составляла 15-100 % ( в среднем 44,0 ± 3,2 % ) и закономерно снижалась по мере увеличения продолжительности жизни . показана взаимозависимость таких параметров популяций , как удельная продукция , естественная смертность , продолжительность жизни и средняя масса особей . трофическая структура соо

## Abstractive methods

### mT5

In [1]:
! git clone https://github.com/IlyaGusev/summarus.git

Cloning into 'summarus'...
remote: Enumerating objects: 1484, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 1484 (delta 98), reused 96 (delta 78), pack-reused 1366
Receiving objects: 100% (1484/1484), 505.42 KiB | 2.63 MiB/s, done.
Resolving deltas: 100% (987/987), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
! python summarus/external/hf_scripts/train.py \
    --config-file "summarus/external/hf_scripts/configs/t5_training_config.json" \
    --checkpoint "drive/MyDrive/t5-finetuned-2/checkpoint-250" \
    --train-file "train_data.json" \
    --val-file "val_data.json" \
    --output-dir "drive/MyDrive/t5-finetuned-2-2" \
    --source-field "article" \
    --target-field "abstract"

2023-04-09 23:41:21.916550: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--cointegrated--rut5-base/snapshots/49ce7aa5a7540620121965a8486f66019f7447d2/config.json
Model config T5Config {
  "_name_or_path": "cointegrated/rut5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class":

In [14]:
%cd summarus/external/hf_scripts

/home/ichuviliaeva/is_project/summarus/external/hf_scripts


In [15]:
with open('t5_gold.txt', "w") as w:
    for p in datasets['test']['abstract']:
        w.write(p.strip().replace("\n", " ") + "\n")

In [16]:
!ls /DATA/ichuviliaeva/project_data/t5_checkpoint/checkpoint-500

config.json		optimizer.pt	   rng_state.pth  trainer_state.json
generation_config.json	pytorch_model.bin  scheduler.pt   training_args.bin


In [17]:
!ls

configs		     predict_extractive.py  requirements.txt	 train.py
dataset.py	     predict.py		    t5_gold.txt		 util.py
extractive_model.py  __pycache__	    train_extractive.py


In [20]:
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [22]:
! python predict.py --input-file "/DATA/ichuviliaeva/project_data/test_data.json" --output-file "t5_predictions.txt" \
  --model-name "/DATA/ichuviliaeva/project_data/t5_checkpoint/checkpoint-500" \
  --model-type "seq2seq_lm" \
  --batch-size 16

/home/ichuviliaeva/miniconda3/envs/is_project/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
0it [00:00, ?it/s]/home/ichuviliaeva/miniconda3/envs/is_project/lib/python3.9/site-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/home/ichuvil

In [28]:
import os
import argparse
import re

import numpy as np

import razdel
import nltk

from collections import Counter
from statistics import mean

from rouge import Rouge
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.chrf_score import corpus_chrf
import torch


def calc_metrics(refs, hyps):
    metrics = dict()
    metrics["count"] = len(hyps)
    metrics["ref_example"] = refs[-1]
    metrics["hyp_example"] = hyps[-1]
    many_refs = [[r] if r is not list else r for r in refs]

    # Calculate BLEU score
    t_hyps = [hyp.split(" ") for hyp in hyps]
    t_refs = [[r.split(" ") for r in rs] for rs in many_refs]
    metrics["bleu"] = corpus_bleu(t_refs, t_hyps)

    # Calculate BLEU score
    rouge = Rouge()
    # Remove empty hypotheses
    idxs = np.where([len(h) > 0 for h in hyps])[0]
    scores = rouge.get_scores(np.array(hyps)[idxs], np.array(refs)[idxs], avg=True)
    metrics.update(scores)

    # Calculate mean abstracts' length
    metrics["length"] = mean([len(h) for h in hyps])

    return metrics


def print_metrics(refs, hyps):
    metrics = calc_metrics(refs, hyps)

    print("-------------METRICS-------------")
    print("Count:\t", metrics["count"])
    print("Ref:\t", metrics["ref_example"])
    print("Hyp:\t", metrics["hyp_example"])

    print("BLEU:     \t{:3.1f}".format(metrics["bleu"] * 100.0))

    print("ROUGE-1-F:\t{:3.1f}".format(metrics["rouge-1"]['f'] * 100.0))
    print("ROUGE-2-F:\t{:3.1f}".format(metrics["rouge-2"]['f'] * 100.0))
    print("ROUGE-L-F:\t{:3.1f}".format(metrics["rouge-l"]['f'] * 100.0))

    print("Avg length:\t{:3.1f}".format(metrics["length"]))


def postprocess(ref, hyp, tokenize_after=False, lower=False):
    ref = ref.strip()
    hyp = hyp.strip()
    if tokenize_after:
        hyp = hyp.replace("@@UNKNOWN@@", "<unk>")
        hyp = " ".join([token.text for token in razdel.tokenize(hyp)])
        ref = " ".join([token.text for token in razdel.tokenize(ref)])
    if lower:
        hyp = hyp.lower()
        ref = ref.lower()
    return ref, hyp

In [29]:
from tqdm.notebook import tqdm

def calcs(predictions_file, targets_file):
    preds = []
    targets = []
    with open(predictions_file, "r") as r:
        for rl in tqdm(r.readlines()):
            preds.append(rl)
    with open(targets_file, "r") as r:
        for rl in tqdm(r.readlines()):
            targets.append(rl)
            
    for i, (target, pred) in enumerate(tqdm(zip(targets, preds))):
        targets[i], preds[i] = postprocess(target, pred, tokenize_after=True, lower=True)
        
    print_metrics(targets, preds)

In [30]:
calcs("t5_predictions.txt", "t5_gold.txt")

  0%|          | 0/3810 [00:00<?, ?it/s]

  0%|          | 0/3810 [00:00<?, ?it/s]

0it [00:00, ?it/s]

-------------METRICS-------------
Count:	 3810
Ref:	 в статье дан анализ структуры технологического процесса авиационнохимической обработки полей , в частности , процесса технологической подготовки , концепций его оптимизации . i
Hyp:	 в статье представлены результаты исследования рабочих операций технологического процесса , а также расчета времени , отводимого на проведение вспомогательной операции тподг ( раб ) и тпер . 1 + поворота , ( 2 ) где тпер . 1 - время перелета сла от взлетнопосадочной полосы до обрабатываемого участка ; шоворота - поворот и выход сла на обрабатываемый участок .
BLEU:     	9.0
ROUGE-1-F:	26.0
ROUGE-2-F:	13.5
ROUGE-L-F:	24.5
Avg length:	483.9
